In [8]:
import re
import requests
import numpy as np
import pandas as pd
from itertools import chain
from bs4 import BeautifulSoup

In [69]:
eu_cities = ['Badajoz','Kiev','Orenburg','Hannover','Birmingham, U.K.','Turin','Milan','Chemnitz','Minsk','Dublin','Le Mans','Gdansk','Botosani','Bonn','Volgograd','Bern','Modena','Aalborg','Cambridge',
     'Lille','Kremenchuk','Glasgow','Sofia','Coventry','Gaziantep','Ankara','Balti','Newcastle-upon-Tyne','Erfurt','Helsinki','Chernivtsi','Lucerne','Warsaw','Tolyatti','Basel',
 'Saint-Étienne','Zhytomyr','Freiburg','Craiova','Strasbourg','Novosibirsk','Metz','Jerusalem','Hamburg','Skopje','Frankfurt','Tartu','Exeter',
 'Bournemouth','Pleven','Munich','Tekirdag','Yevpatoriya','Nuremberg','Koszalin','Verona','Toulouse','Duisburg','Madrid','Arad','Novi Sad','St Petersburg',
 'Izhevsk','Cartagena','Rouen','Denizli','Salzburg','Trieste','Tarsus','Bila Tserkva','Huelva','Split','Liège','Rivne','Grenoble','Nice',
 'Ancona','Cannes','Bacau','Batman','Kiruna','Katowice','Oldenburg','Nis','Bergen','Amiens','Dijon','Bytom','Kazan','Braila','Marseille','Amsterdam',
 'Lisbon','Reims','Uppsala','Tirana','Ruse','Vigo','Karlsruhe','Ostrava','Chrisinau','Blackpool','Pinsk','Plovdiv','Bristol','Cologne','Angers',
 'Munster','Kalamata','Malatya','Martin','Erzurum','Granada','Orsha','Antalya','Patras','Limoges','Augsburg','Sarajevo','Seville','Magdeburg',
 'Mulhouse','Galway','Bordeaux','Edinburgh','Debrecen','Siirt','Belfast','Como','Sheffield','Düsseldorf','Dubrovnik','Ulm','Kayseri','Bologna',
 'Chernihiv','Thessaloniki','Potsdam','Luxembourg','Odense','Trondheim','Inverness','Erzincan','Le Havre','Toulon','Foggia','Rennes',
 'Oporto','Nizhny Novgorod','Wuppertal','Bratislava','Bielefeld','Oslo','Saratov','Samara','Essen','MalmÃ¶','Catania','Szeged','Heidelberg',
 'Rome','GÃ¶teborg','Braga','Burgos','Belgrade','Berlin','Makiyivka','Sibiu','Turku','Antwerp','Poznan','Caen','Manchester','Kryvyy Rih',
 'Málaga','Messina','Tel Aviv','Eindhoven','Lyon','Palermo','Groningen','Geneva','Murcia','Hanover','Bursa','Bialystok','Karaman','Sumy',
 'Stara Zagora','Cherkasy','Breda','Malta','Athens','Barnaul','Lvov','The Hague','Montpellier','Swansea','Elblag','Tampere','Rotterdam', 'Florence','Aberdeen',
 'Torino','Kherson','Elbasan','Burgas','Mazyr','Podgorica','Vienna','Rostov-na-Donu','Horlivka','Bochum','Bodo','Timisoara','Leeds','Venice',
 'Eskisehir','Gent','Nantes','Bucharest','Klaipeda','Padova','Mannheim-Heidelberg','Brugge','Kielce','Utrecht','Naberezhnye Tchelny','Stuttgart',
 'Zürich','London','Bradford','Cork','Linz','Corunna', 'Zagreb','Daugavpils','Manisa','Naples','Rijeka','Budapest','Hrodna','Valencia',
 'Yekaterinburg','Savona','Oulu','Stockholm','Bath & NE Somerset','Abisko','Andorra','Kaliningrad','Leipzig','Maastricht','Leuven','Albacete',
 'Chisinau','Tallinn','Sivas','Pescara','Perm','Kaunas','Riga','Dortmund','Bydgoszcz','Ordu','Brno','Zurich','Barcelona','Waterford',
 'Brussels','Bremen','Reykjavik','Bergamo','Constanta','Oviedo','Perugia','Aachen','Zonguldak','Kosice','Santander','Bilbao','Dresden','Copenhagen', 'Brest','Tomsk','Kiel','Wroclaw',
 'Pamplona','Genoa','Malaga','Kingston-upon-Hull','Paris','Samsun','Ingolstadt','Trabzon','Darmstadt','Krasnoyarsk','Moscow','Marbella','Vilnius',
 'Adana','Bari','Cosenza','Trikala','Graz','Dundee','Ljubljana','Omsk','Baia Mare','Vladivostok','Istanbul','Parma','Innsbruck','Prague',
 'Birmingham','Malmö','Perpignan','Rostock','Kraków','Edirne','Salamanca','Zaragoza','Gyor','Gothenburg','Stavanger','Cardiff',
 'Liverpool','Algeciras','Gijón']

In [ ]:
usa_cities= ['Marysville','Perris','Cleveland','Worcester','Columbia','Waterbury','Eagan','Southfield','Lafayette','Boise City','Elkhart','Buena Park',
 'Marietta','Parker','West Palm Beach','Salem','Aurora','Leesburg','Doral','Westminster','Lubbock','Overland Park','Jackson','Gastonia','Daytona Beach',
 'Kansas City','Noblesville','Southaven','Chino','Gaithersburg','Fountain Valley','Frederick','Greenville','DeKalb','West Allis','Thornton',
 'West Des Moines','Fall River','Tulare','Chula Vista','Coral Springs','Concord','Smyrna','Apopka','Grand Forks','Lancaster','Naperville','Hollywood',
 'Rogers','New Orleans','Lakewood','Oak Lawn','Caldwell','Portland','Concord','Burien','St. Charles','Waukesha','Flagstaff',
 'Puyallup','Schenectady','Denver','Springfield','Campbell','Moreno Valley','Kettering','Lincoln Park','Lake Havasu City','North Little Rock',
 'Reno','Littleton','Prescott','Dublin','Texarkana','Portland','Brooklyn Park','West Haven','El Centro','The Colony','Flint','Pittsburg','Buckeye',
 'Brea','Indio','Scottsdale','Columbus','Lancaster','Lincoln','Trenton','Boston','Glendora','Rochester','Florissant','Annapolis','Chapel Hill',
 'Lawrence','Calexico','Calumet City','Missoula','Clovis','Pearland','Gilbert','Wausau','Raleigh','Orange','Walnut Creek','Bremerton','Porterville',
 'Marana','Tallahassee','Conroe','Blacksburg','San Antonio','Hartford','Waterloo','Binghamton','Meridian','Fond du Lac','Baytown','Cary',
 'Weymouth Town','Tulsa','Mission','Cuyahoga Falls','Malden', 'Allen','Tucson','Franklin','Shakopee','Jacksonville','Atlantic City','Erie','Merced',
 'Abilene','Yorba Linda','Greenville','Huntersville','St. Clair Shores','Harrisburg','San Buenaventura (Ventura)','Fremont','Pueblo','Chelsea',
 'Belleville','Minneapolis','Bartlett','Wichita Falls','Missouri City','St. Louis','Pharr','Albuquerque','Athens-Clarke County','South Jordan','North Las Vegas',
 'Pomona','Eastvale','Grove City','Lawrence','Midland','Oklahoma City','Oro Valley','Mesa','Norwalk','Kentwood','Brookfield','Rockwall','Mishawaka',
 'Tustin','St. Cloud','Passaic','Escondido','San Bruno','Berkeley','San Clemente','Lawton','Eau Claire','Maricopa','DeSoto','Fort Wayne',
 'Shoreline','Medford','La Habra','Albany','Daly City','Taylor','Coon Rapids','San Mateo','Orlando','Casper','Bedford','Wilmington','Pinellas Park',
 'North Miami Beach','Kansas City','Fitchburg','Wheeling','Newton','North Miami','Norwich','Newark','Alexandria','Blaine','Harlingen','Lompoc',
 'Denton','Galveston','Greenwood','Salinas','Miami Gardens','Huntington','Laredo','Melbourne','Haltom City','Palm Bay','Wilson','Bountiful','Ceres',
 'Parma','Rapid City','Bell Gardens','Commerce City','Springfield','Fort Collins','San Leandro','Vista','Yonkers','Cincinnati',
 'Hagerstown','Sanford','Union City','Toledo','Sierra Vista','Newport News','Hammond','Buffalo Grove','Sugar Land','Huntsville','Lodi','Edmonds',
 'Duncanville','Rocky Mount','Pontiac','National City','Hayward','Niagara Falls','Sparks','Hanford','Cypress','Decatur','Woonsocket','Mobile',
 'Vallejo','Arcadia','Collierville','Jefferson City','Longview','Chandler','Arvada','Alhambra','Kokomo','Oakland Park','Indianapolis','Boynton Beach','Apache Junction',
 'Mount Prospect','Cranston','Lafayette','Olathe','Johns Creek','Greensboro','Kalamazoo','Tigard','Jacksonville','Citrus Heights','Tuscaloosa',
 'Ankeny','Hawthorne','Hoover','Youngstown','Madison','Redlands','Cape Girardeau','Nampa','Washington','Wheaton','Lancaster','Rialto',
 'Fargo','Peoria','Omaha','Rancho Santa Margarita','Manhattan','Palm Springs','San Diego','Santa Rosa','Lowell','Broken Arrow','Kent','Nashville',
 'Hackensack','Broomfield','Durham','Wichita','Cathedral City','Lakeville','Elmhurst','Weslaco','Greenfield','Sunnyvale','Santa Cruz','Kenosha',
 'State College','Mansfield','Quincy','Pompano Beach','San Angelo','Bloomington','Tracy','Spartanburg','Nashua','Macon','Hempstead','Camden',
 'Palm Desert','Streamwood','Chesapeake','Dublin','Camarillo','Muskegon','Elizabeth','Anchorage','Minot','Davie','McAllen','McKinney',
 'Killeen','Findlay','Oakland','Mountain View','Lake Elsinore','Amarillo','Provo','Atlanta','Monterey Park','Strongsville',
 'Madera','Rancho Palos Verdes','Burnsville','Rowlett','Roswell','Montgomery','Logan','Woburn','Allentown','Mount Pleasant','Columbus','Buffalo',
 'New Berlin','North Richland Hills','Lima','Sacramento','Fort Smith','East Providence','Urbandale','Mount Vernon','Lakewood','Henderson','Covina',
 'Stamford','Bloomington','Kingsport','Spokane','Kennewick','Philadelphia','Bellingham','Visalia','La Crosse','Council Bluffs','North Lauderdale','Dallas',
 'Antioch','Sterling Heights','Deerfield Beach','Fishers','Cheyenne','Murrieta','Lenexa','Newport Beach','Riverton','Yakima','Dayton','Gulfport',
 'Long Beach','Chico','Hattiesburg','Eugene','Colton','Montebello','Pleasanton','Everett','Hutchinson','Fairfield','Avondale','Euclid','Battle Creek',
 'Bloomington','Rock Island','Beaverton','Kenner','Clovis','Troy','Milpitas','Des Plaines','Placentia','Taunton','Montclair','Johnson City','Savannah',
 'Warren','Vancouver','Sunrise','Muncie','Lehi','San Francisco','Fontana','Frisco','Chino Hills','Poway','Huber Heights','New Brunswick','Norfolk',
 'Hillsboro','San Jacinto','Pensacola','Hamilton','Florence','San Bernardino','Peachtree Corners','Victorville','Gary','Casa Grande','Morgan Hill',
 'Dubuque','Syracuse','Clarksville','Lakeland','Valley Stream','Dover','Moorhead','Culver City','Palatine','Seattle','Covington','Elk Grove','Shreveport',
 'Ogden','Harrisonburg','Inglewood','Pasadena','Lawrence','Lake Oswego','Petaluma','Springfield','Folsom','Prescott Valley','Las Cruces','Bozeman','Schaumburg','Warren','St. Peters',
 'Ames','Surprise','Richardson','Midland','San Marcos','Wilkes-Barre','Appleton','Shawnee','Mentor','Livermore','Charlotte','Owensboro','Jupiter','Rohnert Park',
 'Chicago','Billings','Wauwatosa','Lombard','New Haven','York','Knoxville','Elyria','Barnstable Town','Murray','Alameda','Manteca','Maplewood',
 'Medford','Murfreesboro','Yuba City','Shelton','Cedar Falls','Germantown','Scranton','Auburn','Manassas','Vacaville','Santa Monica','Suffolk',
 'Rochester Hills','La Quinta','Charleston','Delano','St. Cloud','Charleston','Kannapolis','Corona','Rocklin',"Lee's Summit",'Palm Coast','Sandy',
 'Bentonville','Santa Clarita','Altamonte Springs','New Braunfels','Memphis','Irving','Alexandria','Hesperia','Holyoke','Dunwoody','Skokie',
 'Kearny','Bethlehem','Napa','Aurora','Duluth','Roanoke','Bend','Apple Valley','Waltham','Fort Myers','Tampa','Upland','Danville','Beloit',
 'Plantation','Colorado Springs','Modesto','Grand Rapids','Oak Park','Sarasota','Jurupa Valley','Bellflower','Boulder','Santa Fe','Coachella',
 'Marlborough','Riverside','Palmdale','Novato','Layton','Bolingbrook','Mission Viejo','Glenview','Lincoln','Goose Creek','Little Rock','Santa Ana','White Plains','Idaho Falls','Bellevue','Roseville','Perth Amboy','Peabody','Grand Island','Columbia','Racine','Carmel',
 'Portage','Middletown','Woodland','Revere', 'Euless', 'Arlington Heights', 'Hemet','High Point', 'Newark', 'Midwest City', 'San Marcos', 'Port Orange', 'Lakewood',
 'Westerville', 'Rockford', 'El Cajon', 'St. Petersburg', 'Downers Grove','Milford', 'Albany','Las Vegas', 'Pasadena', 'Port St. Lucie', 'Cedar Rapids',
 'Ormond Beach', 'Quincy', 'Miramar', 'El Paso', 'Topeka', 'Rosemead', 'Keizer', 'Minnetonka', 'Akron', 'Edinburg', 'Rancho Cucamonga',
 'North Charleston','Louisville', 'Watsonville', 'Coconut Creek','Bradenton', 'Carlsbad', 'Texas City', 'Redding', 'Wyoming', 'Lorain', 'Roy', 'Fort Lauderdale',
 'Warner Robins', 'Clearwater', 'Cutler Bay', 'Florence', 'Terre Haute', 'Temple', 'Beavercreek', 'Sioux Falls', 'Columbus', 'Margate', 'Leominster',
 'Warwick','Bakersfield', 'Miami','Lake Charles', 'Richland', 'Hickory', 'Delray Beach', 'Wellington','Bullhead City', 'Weston', 'Troy', 'New Rochelle',
 'Stockton','Royal Oak', 'Titusville', 'Laguna Niguel', 'Spokane Valley', 'Costa Mesa','Grand Prairie', 'Portsmouth','Lacey', 'Apex', 'Sioux City',
 'Encinitas', 'Spanish Fork', 'Huntington Park', 'Eden Prairie', 'Peoria', 'Tempe', 'Wilmington', 'Taylorsville', 'Westland', 'Mesquite', 'Burlington', 'Garland',
 'Richmond','Bismarck','Carson City','Muskogee', 'Augusta-Richmond County','Fayetteville' ,'Albany','Sayreville', 'Newark', 'Diamond Bar', 'Decatur',
 'Chattanooga', 'Jeffersonville', 'Bartlett', 'Baldwin Park', 'Miami Beach', 'Jackson','Port Arthur', 'San Luis Obispo', 'Burleson', 'St. Joseph',
 'Redondo Beach', 'Houston', 'Temecula', 'Ann Arbor', 'Beaumont', 'East Lansing', 'Santee', 'Tamarac', 'Bellevue', 'Pembroke Pines', 'Springdale',
 'Oxnard', 'Odessa', 'Stillwater', 'Brockton', 'Bayonne', 'Paramount', 'Moline', 'Meridian', 'Cleveland', 'Jersey City', 'St. Paul', 'Deltona',
 'Freeport', 'Iowa City', 'Anaheim', 'Apple Valley','Great Falls', 'West Valley City', 'Hampton', 'Georgetown', 'Orem', 'Monrovia', 'Keller',
 'South San Francisco', 'Plymouth', 'South Bend', 'Edmond', 'Des Moines', 'Bonita Springs', 'Draper', 'Whittier', 'Livonia', 'Milwaukee', 'Auburn',
 'Sumter', 'Beverly', 'Renton', 'Centennial', 'Richmond', 'Anderson', 'Beaumont', 'Pflugerville', 'Hoboken', 'Lexington-Fayette', 'Norwalk','Yuma',
 'Highland', 'Carol Stream', 'Joliet', 'Rock Hill', 'Rancho Cordova', 'Dearborn Heights', 'Honolulu', 'Victoria','Attleboro','Wylie','Carrollton',
 'Plainfield', 'Hoffman Estates', 'Methuen','North Port', 'Rockville', 'Burbank','Fullerton', 'Oshkosh', 'Huntsville', 'Farmington Hills', 'Pico Rivera',
 'Cerritos', 'La Puente', 'Grand Junction','Enid', 'San Jose', 'Round Rock', 'Maple Grove', 'Norman', 'Homestead', 'Friendswood', 'Irvine',
 'Pocatello', 'Pacifica', 'Sheboygan','Bossier City', 'Cedar Park', 'Sandy Springs', 'Champaign', 'Clifton', 'Moore', 'Blue Springs', 'Lancaster',
 'Bryan', 'Hurst', 'Cape Coral', 'Cupertino', 'Pittsfield','Gilroy', 'Manchester', 'Virginia Beach', 'Salina', 'Somerville', 'Largo',
 'Fresno', 'Phenix City', 'Berwyn', 'Palm Beach Gardens', 'Pawtucket', 'San Gabriel', 'Winston-Salem', 'Alpharetta', "O'Fallon", 'Redwood City', 'Union City',
 'Hallandale Beach','Mankato', 'New York', 'Simi Valley','West Covina', 'Castle Rock', 'Valdosta', 'Fort Worth', 'Carson', 'Evansville', 'Santa Maria',
 'Brentwood', 'La Mesa', 'Westminster', 'Reading','Turlock', 'Greeley', 'Tyler', 'Winter Garden', 'Waco', 'Martinez', 'Thousand Oaks',
 'Azusa', 'Bridgeport', 'St. Louis Park', 'Chesterfield', 'Greenacres', 'Pasco', 'Torrance', 'Madison','Janesville', 'Waukegan', 'Bowling Green', 'Arlington',
 'Lewisville', 'Farmington', 'Oakley', 'Palo Alto','Springfield', 'New Bedford', 'Redmond', 'Brentwood', 'Davis', 'La Mirada', 'Smyrna', 'Altoona', 'Dothan',
 'East Orange', 'Tinley Park', 'Biloxi', 'Urbana', 'Garden Grove', 'Lynwood', 'Northglenn', 'Canton','Brookhaven', 'Baton Rouge', 'Bristol',
 'Independence', 'Lansing', 'Orland Park','Vineland', 'Green Bay', 'Evanston','Ocoee', 'Los Angeles', 'Federal Way', 'West Jordan', 'Yucaipa',
 'Pittsburgh','Downey', 'Cicero', 'South Gate','Panama City', 'Joplin', 'League City', 'Salt Lake City', 'Fort Pierce', 'Phoenix', 'Corvallis',
 'Cedar Hill','Chicopee', 'Ocala', 'Roswell', 'Park Ridge', 'Concord', 'Olympia', 'Huntington Beach', 'Corpus Christi', 'Haverhill', 'Hanover Park',
 'Cambridge', 'Glendale', 'Stanton', 'Summerville', 'Davenport', 'Boca Raton', 'New Britain', 'Roseville', 'Cleveland Heights', 'Gresham', 'Lauderhill',
 'Hendersonville', 'Santa Clara', 'Oceanside','Lake Forest', 'Novi', 'West New York', 'Everett', 'Linden', 'Asheville', 'Compton', 'Aliso Viejo',
 'Menifee', 'Loveland', 'Carpentersville', 'Providence', 'Hialeah', 'San Ramon', 'Ontario', 'Kirkland', 'St. George', 'Hilton Head Island', 'Glendale',
 'Meriden','Gardena', 'Westfield', 'Elgin', 'Plano', 'Longmont', 'Coppell', 'Lakewood', 'Jonesboro', 'College Station', 'Dearborn', 'Utica',
 'Monroe', 'Baltimore', 'Springfield', 'Pine Bluff', 'Plainfield', 'Lynn', 'Woodbury', 'Gainesville', 'West Sacramento','Sammamish', 'Aventura',
 'Fayetteville','San Rafael','Conway', 'Tacoma','Middletown', 'Burlington', 'Twin Falls', 'Sherman', 'Lynchburg', 'Bowie', 'Danbury',
 'Charlottesville','Coral Gables',"Coeur d'Alene", 'El Monte', 'Crystal Lake', 'Danville', 'Goodyear', 'Paterson', 'Salem', 'Kissimmee', 'Saginaw',
 'Brownsville','Flower Mound','Grapevine','Edina', 'Detroit', 'Fairfield', 'Mansfield', 'Rochester', 'Birmingham', 'Austin', 'Santa Barbara', 'Addison', 'Rio Rancho', 'Normal', 'Romeoville']

In [10]:
def create_frame(currency, ls):
    
    total = pd.DataFrame()
    
    for city in ls:
        new =  []

        page = requests.get("https://www.numbeo.com/cost-of-living/in/" + city)
        soup = BeautifulSoup(page.content)
        data = soup.find_all("table",{"class":"data_wide_table new_bar_table"})
        data_cost = soup.find_all("span",{"class":"emp_number"})
        try:
            cost_index = re.search('>(.*)\<', str(data_cost[-1])).group(1)
        except IndexError:
            cost_index = '0'
        
        l = [item.text.split(currency) for item in data]
        newlist = list(chain(*l))

        new.append('City' + '  '  + city)
        new.append('Cost_index' + '  ' + cost_index)

        try:
            [new.append(re.search('\n(.*)\xa0', newlist[i]).group(1)) for i in range(len(newlist))]            
        except AttributeError as e:
            pass
            
        if len(new) != 56:
            pass
        
        else:
            for i in range(len(new)):
                total = total.append(pd.DataFrame({'':new[i].split('  ')[0], 'Cost':new[i].split('  ')[1]},index=[0]), ignore_index=True)
            
            headers = total.T.iloc[0]
            headers = pd.DataFrame(headers[:56])
            
    values = []
    n = len(total)/56
    df_list = np.vsplit(total, n)

    for i in df_list:
        values.append(i['Cost'].tolist())
    
    v = pd.DataFrame(values)
    zz = pd.DataFrame(headers).T
    xx = pd.concat([zz,v])
    new_header = xx.iloc[0]
    xx = xx[1:]
    xx.columns = new_header
    
    final = xx.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    final = final.drop(['Water (0.33 liter bottle)'], axis = 1)
    
    zz = final['City']
    z5 = final.loc[:, final.columns != 'City']
    z5 = z5.replace(',', '', regex=True)
    z39 = z5.apply(pd.to_numeric, errors='coerce')
    df = pd.concat([zz, z39],axis=1)
    
            
    return df

In [11]:
df = create_frame('€', z)

In [15]:
df.to_pickle('eu.pkl')

In [71]:
df_eu = pd.read_pickle('eu.pkl')

In [72]:
df_eu

,City,Cost_index,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),"Milk (regular), (1 liter)",...,"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax)
0,Turin,65.63,15.0,60.0,8.00,4.00,4.00,1.41,1.98,1.10,...,31.92,76.41,120.91,568.75,420.31,1107.24,765.17,3435.42,1976.00,1413.44
1,Milan,75.78,15.0,60.0,8.00,5.00,5.00,1.51,2.54,1.23,...,35.83,90.99,138.49,1167.00,748.08,2507.87,1514.29,8549.13,3786.26,1651.84
2,Chemnitz,0.00,8.5,40.0,7.75,3.00,3.75,3.00,2.08,0.68,...,24.00,68.75,88.75,385.00,332.00,850.00,675.00,3000.00,1516.67,1962.50
3,Dublin,77.85,15.0,62.5,8.50,5.75,6.00,3.29,1.86,0.97,...,34.70,83.30,97.38,1709.85,1434.79,3035.56,2372.22,5587.01,4026.59,2490.81
4,Bonn,66.01,9.5,55.0,7.50,3.50,3.50,2.61,2.06,0.74,...,35.50,72.88,100.43,784.62,586.15,1507.69,1138.46,7433.33,4933.33,2431.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Graz,72.49,10.0,45.0,7.90,3.80,3.50,3.05,2.36,1.15,...,28.97,83.62,119.69,589.26,440.75,1024.12,803.75,3691.79,2907.65,1911.25
109,Ljubljana,54.69,8.5,40.0,5.00,2.70,3.50,1.56,2.26,0.92,...,28.77,78.08,89.54,597.31,464.62,1104.44,819.88,3754.05,2729.66,1184.69
110,Parma,70.83,20.0,60.0,8.50,4.50,4.00,1.48,2.33,1.20,...,38.75,92.50,128.75,560.00,433.33,1075.00,787.50,3428.57,2050.00,1494.44
111,Innsbruck,73.94,12.5,60.0,7.90,3.50,3.75,2.86,2.91,1.12,...,35.75,82.50,105.00,779.17,648.33,1527.27,1175.00,6114.44,4622.22,1892.42


In [5]:
df = create_frame('$', x)

In [7]:
df.to_pickle('usa.pkl')

In [73]:
df_usa = pd.read_pickle('usa.pkl')

In [70]:
def convert_currency(df):
    
    rate = 0.91
    test = df_usa.drop(['Cost_index'],axis=1)
    s = round(test.select_dtypes(include=['number']) * convert_rate, 2)
    final = pd.concat([df_usa.City, df_usa.Cost_index, s], axis=1)
    
    return final

In [74]:
df_usa_f = convert_currency(df_usa)

In [75]:
df_usa_f

,City,Cost_index,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),"Milk (regular), (1 liter)",...,"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax)
0,Cleveland,77.28,13.65,50.05,7.28,3.64,4.55,3.54,1.81,0.66,...,26.86,73.52,74.14,1117.00,731.18,2073.75,1342.25,946.40,697.49,2951.22
1,Marietta,0.00,11.38,36.40,6.37,3.64,5.46,3.78,1.71,0.56,...,31.22,70.48,96.85,1214.85,838.11,1592.50,1410.50,3428.31,1616.21,4722.29
2,Lubbock,44.64,7.74,33.49,6.37,3.08,3.64,4.04,1.41,0.70,...,31.85,51.32,72.80,722.31,576.33,1091.02,971.42,1074.11,753.76,2580.93
3,Greenville,0.00,11.83,45.50,6.37,4.55,5.46,3.84,1.43,0.72,...,31.22,70.48,96.85,1147.61,912.37,1716.00,1256.94,1453.60,1045.64,3483.43
4,Portland,82.05,13.65,54.60,6.82,4.55,4.78,3.93,1.80,0.83,...,36.36,72.55,86.94,1431.76,1109.22,2563.82,1816.01,3755.08,2412.18,3425.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Tacoma,75.52,11.83,45.50,8.19,3.64,5.00,3.89,1.63,0.76,...,34.40,73.08,98.46,1223.40,1071.45,2044.25,1616.77,2828.63,2165.45,3453.73
131,Burlington,98.31,18.20,68.25,8.19,4.78,5.92,4.48,1.97,0.96,...,33.36,68.43,83.04,1205.75,910.00,2487.33,2129.40,3212.81,1922.30,4575.28
132,Detroit,63.92,10.01,40.95,6.37,3.18,4.55,3.37,1.67,0.57,...,29.48,73.66,80.76,1300.77,793.58,2229.50,1297.73,511.52,547.10,2240.64
133,Rochester,73.50,13.65,54.60,7.28,4.54,6.36,3.80,1.87,0.61,...,30.18,57.05,77.35,891.91,801.35,1264.90,1238.25,1070.66,761.57,2638.57
